## Ranking Model 학습시키기

### 1. csv 파일 준비하기 (모델 학습을 위한 데이터)

In [7]:
import mysql.connector as sql

import pandas as pd

host = '127.0.0.1'
port = 3306
username = 'root'
password = '1111'
database_name = 'dealicious'

db_connection = sql.connect(host=host, database=database_name, user=username, password=password)

retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)

retrievals_df.head(3)

/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_64635/2380683196.py:13: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  retrievals_df = pd.read_sql('SELECT * FROM rec_retrievals', con=db_connection)


,t_dat,customer_id,article_id,price,sales_channel_id,month_sin,month_cos,age,garment_group_name,index_group_name
0,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,710390001,0.019051,1,-0.866025,-0.5,20.0,Skirts,Divided
1,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,633130019,0.016932,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided
2,1537401600000000000,0095c9b47fc950788bb709201f024c5338838a27c59c02...,671057002,0.008458,1,-0.866025,-0.5,20.0,Jersey Fancy,Divided


In [8]:
from sklearn.model_selection import train_test_split

train_df, test_df = train_test_split(retrievals_df, test_size=0.2, random_state=42)

In [9]:
train_df["article_id"] = train_df["article_id"].astype(str) # to be deleted
test_df["article_id"] = test_df["article_id"].astype(str)

`USE_TRAIN` 를 각각 False, True 로 두고 아래의 코드 한번씩 실행
- ranking_train.csv
- ranking_test.csv 

In [10]:
USE_TRAIN = True

split, df = ("train", train_df) if USE_TRAIN else ("validation", test_df)
df['article_id'] = df['article_id'].astype(str)

ds_name = f"ranking_{split}.csv"

In [11]:
# positive_pairs : 실제 구매했던 (고객, 상품) 쌍
query_features = ["customer_id", "age", "month_sin", "month_cos"]

positive_pairs = df[query_features + ["article_id"]].copy()
positive_pairs

,customer_id,age,month_sin,month_cos,article_id
15415,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001
829,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013
14449,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003
5727,35be9d9171c603488f94b78402f2e8b8b9f4244fa84341...,20.0,-0.866025,-5.000000e-01,673677003
5814,4ca7eafddf27a6159ee1eb842ea20057ae60dcdf6f583d...,29.0,-0.866025,-5.000000e-01,399223024
...,...,...,...,...,...
11284,b80df3e553cfa079e8f82b7d2e5fa2140492d9a5474768...,35.0,-1.000000,-1.836970e-16,519749036
11964,a1137ac03d00a3f98ac4997d4c1c7bf9dcdcc707b8baf2...,47.0,-1.000000,-1.836970e-16,658841001
5390,e8f0fd33c39b1c5558eef13401c55f80898bb8f8eca19d...,38.0,-0.866025,-5.000000e-01,537688010
860,e9975722496e741f7de041a4093b991e34ff5a7513c554...,46.0,-0.866025,-5.000000e-01,649551006


In [12]:
# negative_paris : 임의로 샘플링한 구매 x (고객, 상품) 쌍
n_neg = len(positive_pairs)*10

negative_pairs = positive_pairs[query_features]\
    .sample(n_neg, replace=True, random_state=1)\
    .reset_index(drop=True)

negative_pairs["article_id"] = positive_pairs["article_id"]\
    .sample(n_neg, replace=True, random_state=2).to_numpy()

negative_pairs

,customer_id,age,month_sin,month_cos,article_id
0,21b4053393d49a39f9c950935af68ae16545e59c925ad2...,64.0,-1.000000,-1.836970e-16,562761001
1,895576481a1095ad66ab3279483f4323724e9d53d9f089...,43.0,-0.866025,-5.000000e-01,380609008
2,a1ed419b99e97c261727c228a79ca370423d68195613c3...,53.0,-1.000000,-1.836970e-16,685383002
3,311198a56b7e9f98913c8a0080e51f6039374e80fb71e6...,22.0,-0.866025,-5.000000e-01,452851006
4,041f6c7b3be2bd8e8f34a007fee89e4db4e5844c5a0271...,23.0,-0.866025,-5.000000e-01,633130020
...,...,...,...,...,...
147835,2ce30f36fd18893c20ab9fbc9a4669ab7297d61f1e1eff...,42.0,-0.866025,-5.000000e-01,636455009
147836,c49107cfb46bdfb7233f704ba74e116cb37c49becdf04f...,30.0,-0.866025,-5.000000e-01,661162002
147837,db4e50337cb9db6d7face32c50dbd9dc85eab1f18f253d...,33.0,-1.000000,-1.836970e-16,438333016
147838,0b2c608f0e2fe96b66eb29ccf000e1885cbc654594e8d0...,28.0,-1.000000,-1.836970e-16,618387001


In [13]:
# labeling 하기 1 = 구매 o, 0 = 구매 x
import pandas as pd

# Add labels.
positive_pairs["label"] = 1
negative_pairs["label"] = 0

# Concatenate.
ranking_df = pd.concat([positive_pairs, negative_pairs], ignore_index=True)
ranking_df.head()

,customer_id,age,month_sin,month_cos,article_id,label
0,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001,1
1,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013,1
2,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003,1
3,35be9d9171c603488f94b78402f2e8b8b9f4244fa84341...,20.0,-0.866025,-5.000000e-01,673677003,1
4,4ca7eafddf27a6159ee1eb842ea20057ae60dcdf6f583d...,29.0,-0.866025,-5.000000e-01,399223024,1


In [14]:
# Merge with item features.
item_df = pd.read_sql('SELECT * FROM rec_articles', con=db_connection)
item_df.drop_duplicates(subset="article_id", inplace=True)
ranking_df = ranking_df.merge(item_df, on="article_id")

ranking_df.head(3)


/var/folders/fx/b8s33_bs1nx_2nnz6h7w64qm0000gn/T/ipykernel_64635/456510631.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  item_df = pd.read_sql('SELECT * FROM rec_articles', con=db_connection)


,customer_id,age,month_sin,month_cos,article_id,label,product_code,prod_name,product_type_no,product_type_name,...,department_no,department_name,index_code,index_name,index_group_no,index_group_name,section_no,section_name,garment_group_no,garment_group_name
0,9ab3d0b8dfdfafd08b8a23db28a893f748ffce9d269415...,51.0,-1.000000,-1.836970e-16,710899001,1,710899,Harper Turtleneck,252,Sweater,...,1636,Jersey fancy,A,Ladieswear,1,Ladieswear,15,Womens Everyday Collection,1005,Jersey Fancy
1,e425b828c8fedc7456dcec7b3a167e2eacdfee45a39cd8...,20.0,-0.866025,-5.000000e-01,570003013,1,570003,V-NECK SS SLIM FIT,255,T-shirt,...,5832,Light Basic Jersey,F,Menswear,3,Menswear,26,Men Underwear,1002,Jersey Basic
2,2dc0942a0e854456fd08dc03c065b6e0cee1f5499d7176...,47.0,-1.000000,-1.836970e-16,704148003,1,704148,Flirty Metal Stripe Phone case,78,Other accessories,...,3419,Other items,C,Ladies Accessories,1,Ladieswear,66,Womens Small accessories,1019,Accessories


In [15]:
def exclude_feat(s):
    return s.endswith("_id") or s.endswith("_no") or s.endswith("_code")

features_to_exclude = [col for col in ranking_df.columns if exclude_feat(col)]
features_to_exclude.append("prod_name")

ranking_df.drop(features_to_exclude, axis="columns", inplace=True)
ranking_df.head()

,age,month_sin,month_cos,label,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
0,51.0,-1.000000,-1.836970e-16,1,Sweater,Garment Upper body,Melange,Yellow,Medium Dusty,Yellow,Jersey fancy,Ladieswear,Ladieswear,Womens Everyday Collection,Jersey Fancy
1,20.0,-0.866025,-5.000000e-01,1,T-shirt,Garment Upper body,Stripe,Dark Red,Dark,Red,Light Basic Jersey,Menswear,Menswear,Men Underwear,Jersey Basic
2,47.0,-1.000000,-1.836970e-16,1,Other accessories,Accessories,All over pattern,Dark Green,Dark,Green,Other items,Ladies Accessories,Ladieswear,Womens Small accessories,Accessories
3,20.0,-0.866025,-5.000000e-01,1,Sweater,Garment Upper body,Solid,Dark Blue,Dark,Blue,Knitwear,Ladieswear,Ladieswear,Womens Tailoring,Knitwear
4,29.0,-0.866025,-5.000000e-01,1,Trousers,Garment Lower body,Denim,Blue,Medium Dusty,Blue,Denim Trousers,Divided,Divided,Ladies Denim,Trousers Denim


In [16]:
ranking_df.to_csv(ds_name, index=False)

## 2. 모델 학습

In [17]:
import pandas as pd

X_train = pd.read_csv("ranking_train.csv")
X_val = pd.read_csv("ranking_validation.csv")

y_train = X_train.pop("label")
y_val = X_val.pop("label")

X_train.sample(5)

,age,month_sin,month_cos,product_type_name,product_group_name,graphical_appearance_name,colour_group_name,perceived_colour_value_name,perceived_colour_master_name,department_name,index_name,index_group_name,section_name,garment_group_name
96058,45.0,-1.000000,-1.836970e-16,Dress,Garment Full body,All over pattern,Dark Blue,Dark,Blue,Dresses,Divided,Divided,Divided Collection,Dresses Ladies
77104,32.0,-1.000000,-1.836970e-16,Shirt,Garment Upper body,Check,Dark Green,Medium,Green,Shirt,Menswear,Menswear,Contemporary Smart,Shirts
83083,48.0,-1.000000,-1.836970e-16,Hat/brim,Accessories,Solid,Black,Dark,Black,Gloves/Hats,Ladies Accessories,Ladieswear,Womens Big accessories,Accessories
32446,24.0,-1.000000,-1.836970e-16,Hoodie,Garment Upper body,Solid,Light Beige,Dusty Light,Beige,Basics,Menswear,Menswear,Men Underwear,Jersey Basic
87737,29.0,-0.866025,-5.000000e-01,Bikini top,Swimwear,All over pattern,Dark Grey,Dark,Grey,Swimwear,Lingeries/Tights,Ladieswear,"Womens Swimwear, beachwear",Swimwear


In [18]:
input_example = X_train.sample().to_dict("records")[0].keys()
print(input_example)

dict_keys(['age', 'month_sin', 'month_cos', 'product_type_name', 'product_group_name', 'graphical_appearance_name', 'colour_group_name', 'perceived_colour_value_name', 'perceived_colour_master_name', 'department_name', 'index_name', 'index_group_name', 'section_name', 'garment_group_name'])


In [19]:
from catboost import CatBoostClassifier, Pool

cat_features = list(
    X_train.select_dtypes(include=['string', 'object']).columns
)

pool_train = Pool(X_train, y_train, cat_features=cat_features)
pool_val = Pool(X_val, y_val, cat_features=cat_features)

model = CatBoostClassifier(
    learning_rate=0.2,
    iterations=100,
    depth=10,
    scale_pos_weight=10,
    early_stopping_rounds=5,
    use_best_model=True
)

model.fit(pool_train, eval_set=pool_val)

0:	learn: 0.6882719	test: 0.6894614	best: 0.6894614 (0)	total: 131ms	remaining: 13s
1:	learn: 0.6849878	test: 0.6871911	best: 0.6871911 (1)	total: 186ms	remaining: 9.13s
2:	learn: 0.6827642	test: 0.6859294	best: 0.6859294 (2)	total: 213ms	remaining: 6.9s
3:	learn: 0.6815616	test: 0.6851881	best: 0.6851881 (3)	total: 242ms	remaining: 5.82s
4:	learn: 0.6815206	test: 0.6851888	best: 0.6851881 (3)	total: 270ms	remaining: 5.12s
5:	learn: 0.6815044	test: 0.6851908	best: 0.6851881 (3)	total: 281ms	remaining: 4.4s
6:	learn: 0.6797143	test: 0.6841684	best: 0.6841684 (6)	total: 310ms	remaining: 4.11s
7:	learn: 0.6797142	test: 0.6841686	best: 0.6841684 (6)	total: 319ms	remaining: 3.67s
8:	learn: 0.6787626	test: 0.6837438	best: 0.6837438 (8)	total: 353ms	remaining: 3.57s
9:	learn: 0.6787519	test: 0.6837426	best: 0.6837426 (9)	total: 369ms	remaining: 3.32s
10:	learn: 0.6787367	test: 0.6837443	best: 0.6837426 (9)	total: 384ms	remaining: 3.11s
11:	learn: 0.6785789	test: 0.6836937	best: 0.6836937 (11)

In [20]:
from sklearn.metrics import classification_report, precision_recall_fscore_support

preds = model.predict(pool_val)

precision, recall, fscore, _ = precision_recall_fscore_support(y_val, preds, average="binary")

metrics = {
    "precision" : precision,
    "recall" : recall,
    "fscore" : fscore
}

print(classification_report(y_val, preds))

              precision    recall  f1-score   support

           0       0.93      0.58      0.72     36970
           1       0.12      0.56      0.19      3697

    accuracy                           0.58     40667
   macro avg       0.52      0.57      0.46     40667
weighted avg       0.86      0.58      0.67     40667



In [21]:
feat_to_score = {feature: score for feature, score in zip(
    X_train.columns, model.feature_importances_)}

feat_to_score = dict(
    sorted(
        feat_to_score.items(),
        key=lambda item: item[1],
        reverse=True
    )
)

feat_to_score

{'age': 26.729346427839396,
 'index_group_name': 18.187893427258395,
 'section_name': 8.071870828138666,
 'garment_group_name': 6.544300687135788,
 'index_name': 5.727398961093682,
 'graphical_appearance_name': 4.621645051960063,
 'product_type_name': 4.539617719787247,
 'perceived_colour_master_name': 4.243959563343287,
 'month_cos': 4.1872481897349605,
 'perceived_colour_value_name': 3.9613878960313698,
 'product_group_name': 3.62462828272834,
 'department_name': 3.3160002768809895,
 'month_sin': 3.194131849628481,
 'colour_group_name': 3.0505708384393553}

### 3. 모델 저장

In [28]:
# 로컬에 파일 저장
import joblib

joblib.dump(model, 'model.pkl')

['model.pkl']

In [33]:
# model = joblib.load('ranking_model.pkl')

<bound method CatBoostClassifier.predict of <catboost.core.CatBoostClassifier object at 0x1554afca0>>

In [25]:
# 파일 s3에 업로드
import boto3

AWS_PROFILE = 'default'
BUCKET_NAME = 'jimin-model'
session = boto3.Session(profile_name=AWS_PROFILE)
s3_client = session.client('s3')

In [29]:
s3_client.upload_file('model.pkl', BUCKET_NAME, 'ranking_model/model.pkl')

In [37]:
s3_client.download_file(BUCKET_NAME, 'ranking_model', 'rankig_model_downloaded.pkl')

In [42]:
model = joblib.load('rankig_model_downloaded.pkl')
print(model)

input_sample = [
    {
      "age": 50,
      "month_sin": -0.8660254,
      "month_cos": -0.5,
      "product_type_name": "Trousers",
      "product_group_name": "Garment Lower body",
      "graphical_appearance_name": "Denim",
      "colour_group_name": "Black",
      "perceived_colour_value_name": "Dark",
      "perceived_colour_master_name": "Black",
      "department_name": "Denim Trousers",
      "index_name": "Divided",
      "index_group_name": "Divided",
      "section_name": "Ladies Denim",
      "garment_group_name": "Trousers Denim"
    }
  ]

input_df = pd.DataFrame(input_sample)
result = model.predict(input_df)
result

array([0])